In [26]:
from fisher_py.data.business import Scan
from fisher_py import RawFile
from sklearn.metrics.pairwise import cosine_similarity
import re

In [27]:
raw = RawFile("c:/20240708_chm134_Nadig_MAbPac-04A_Proteo06_Group01_Time00_biorep01_techrep01.raw")

In [28]:
def helper_regex(text):
    match = re.search(rf"{'Full'}\s+(\w+)", text)
    if match:
        return match.group(1)
    return None

In [30]:
def cast_func(scan_number):
    scan_masses = []
    scan_intensities = []
    data_masses = []
    data_intensities = [] 


    raw_scan = Scan.from_file(raw._raw_file_access, scan_number=scan_number)
    precursor = float(re.findall(r'[\d]*[.][\d]+', raw_scan.scan_type)[1])
    scan_masses = raw_scan.preferred_masses
    scan_intensities = raw_scan.preferred_intensities

    for i in range(4000, 20000): 
        data_intensities.append(0) 
        data_masses.append(i/10)

    for j in range(0,len(scan_masses)):
        for i in range(0,len(data_masses)):
            if (i+ 4000 + 1)/10 > scan_masses[j] > (i + 4000)/10: 
                data_intensities[i] = data_intensities[i] + scan_intensities[j]

    # for i in range(0,16000):
    #     if (precursor -10) < data_masses[i] < (precursor +10): data_intensities[i] = 0
        
    return(list(data_intensities))

In [71]:
X = [cast_func(1004), cast_func(1008)]
round(cosine_similarity(X)[1][0], 2)

0.25

In [47]:
raw = RawFile("c:/20240708_chm134_Nadig_MAbPac-04A_Proteo06_Group01_Time00_biorep01_techrep01.raw")
ref_scan = cast_func(10244)
scan_id = []
scan_sim = []
for i in range(1, raw.number_of_scans):
    raw_scan = Scan.from_file(raw._raw_file_access, scan_number=i)
    if str(helper_regex(raw_scan.scan_type)) == 'ms2':
        X = [ref_scan, cast_func(i)]
        result = round(cosine_similarity(X)[1][0], 2)
        print(i, "is", result)
        scan_id.append(i)
        scan_sim.append(result)

197 is 0.0
198 is 0.0
199 is 0.01
201 is 0.0
